<a href="https://colab.research.google.com/github/SeeUSoon93/deepLearning/blob/main/U_Net_dog_01.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [1]:
%cd "/content/drive/MyDrive"

/content/drive/MyDrive


In [2]:
import os
import glob
import pandas as pd
import tensorflow as tf
import numpy as np
import json
import matplotlib.pyplot as plt
import tensorflow as tf

from PIL import Image, ImageDraw
from tensorflow.keras.layers import Input, Conv2D, MaxPooling2D, UpSampling2D, concatenate
from tensorflow.keras.models import Model
from tensorflow.keras.metrics import Precision, Recall, AUC
from tensorflow.keras.losses import BinaryCrossentropy
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint

In [3]:
base_folder = 'colab/data/test/dog'
class_folders = os.listdir(base_folder)

In [4]:
image_paths = []
json_paths = []

for class_folder in class_folders:
    image_files = glob.glob(os.path.join(base_folder, class_folder, '*.jpg'))
    json_files = [file.replace('.jpg', '.json') for file in image_files]

    image_paths.extend(image_files)
    json_paths.extend(json_files)

image_paths = [path.replace('\\', '/') for path in image_paths]
json_paths = [path.replace('\\', '/') for path in json_paths]

df = pd.DataFrame({
    'image_path': image_paths,
    'json_path': json_paths,
    'label': [path.split('/')[-2] for path in image_paths]
})

In [5]:
df

,image_path,json_path,label
0,colab/data/test/dog/A1/IMG_D_A1_004832.jpg,colab/data/test/dog/A1/IMG_D_A1_004832.json,A1
1,colab/data/test/dog/A1/IMG_D_A1_004842.jpg,colab/data/test/dog/A1/IMG_D_A1_004842.json,A1
2,colab/data/test/dog/A1/IMG_D_A1_004836.jpg,colab/data/test/dog/A1/IMG_D_A1_004836.json,A1
3,colab/data/test/dog/A1/IMG_D_A1_004818.jpg,colab/data/test/dog/A1/IMG_D_A1_004818.json,A1
4,colab/data/test/dog/A1/IMG_D_A1_004820.jpg,colab/data/test/dog/A1/IMG_D_A1_004820.json,A1
...,...,...,...
29995,colab/data/test/dog/A6/IMG_D_A6_203691.jpg,colab/data/test/dog/A6/IMG_D_A6_203691.json,A6
29996,colab/data/test/dog/A6/IMG_D_A6_203671.jpg,colab/data/test/dog/A6/IMG_D_A6_203671.json,A6
29997,colab/data/test/dog/A6/IMG_D_A6_203677.jpg,colab/data/test/dog/A6/IMG_D_A6_203677.json,A6
29998,colab/data/test/dog/A6/IMG_D_A6_203679.jpg,colab/data/test/dog/A6/IMG_D_A6_203679.json,A6


In [6]:
def create_mask(image_path, json_path):
    try:
        with Image.open(image_path) as img:
            original_size = img.size
            img = img.resize((256, 256))
            image = np.array(img, dtype=np.float32)

        with open(json_path, 'r', encoding='UTF-8') as file:
            data = json.load(file)

        mask = Image.new('L', original_size, 0)
        draw = ImageDraw.Draw(mask)

        for annotation in data['labelingInfo']:
            if 'polygon' in annotation:
                polygon = [(point['x'], point['y']) for point in annotation['polygon']['location'][0]]
                if polygon:
                    draw.polygon(polygon, outline=1, fill=1)

        mask = mask.resize((256, 256))
        mask = np.array(mask, dtype=np.float32)
        mask = np.expand_dims(mask, axis=-1)

        return image, mask
    except Exception as e:
        empty_image = np.zeros((256, 256, 3), dtype=np.float32)
        empty_mask = np.zeros((256, 256, 1), dtype=np.float32)
        return empty_image, empty_mask

In [7]:
def tf_create_mask(image_path, json_path):
    [image, mask] = tf.numpy_function(create_mask, [image_path, json_path], [tf.float32, tf.float32])
    image.set_shape([256, 256, 3])
    mask.set_shape([256, 256, 1])
    return image, mask

image_paths = df['image_path'].values
json_paths = df['json_path'].values

dataset = tf.data.Dataset.from_tensor_slices((image_paths, json_paths))
dataset = dataset.map(tf_create_mask, num_parallel_calls=tf.data.AUTOTUNE)

dataset_size = len(image_paths)
train_size = int(0.8 * dataset_size)
val_size = dataset_size - train_size
batch_size = 8

dataset = dataset.shuffle(buffer_size=5000).cache().repeat()

train_dataset = dataset.take(train_size)
val_dataset = dataset.skip(train_size)

train_dataset = train_dataset.batch(batch_size)
val_dataset = val_dataset.batch(batch_size)

In [8]:
def unet_model(input_size=(256, 256, 3)):
    inputs = Input(input_size)

    # 인코더
    conv1 = Conv2D(32, 3, activation='relu', padding='same')(inputs)
    conv1 = Conv2D(32, 3, activation='relu', padding='same')(conv1)
    pool1 = MaxPooling2D(pool_size=(2, 2))(conv1)

    conv2 = Conv2D(64, 3, activation='relu', padding='same')(pool1)
    conv2 = Conv2D(64, 3, activation='relu', padding='same')(conv2)
    pool2 = MaxPooling2D(pool_size=(2, 2))(conv2)

    conv3 = Conv2D(128, 3, activation='relu', padding='same')(pool2)
    conv3 = Conv2D(128, 3, activation='relu', padding='same')(conv3)
    pool3 = MaxPooling2D(pool_size=(2, 2))(conv3)

    conv4 = Conv2D(256, 3, activation='relu', padding='same')(pool3)
    conv4 = Conv2D(256, 3, activation='relu', padding='same')(conv4)

    # 디코더
    up5 = UpSampling2D(size=(2, 2))(conv4)
    merge5 = concatenate([conv3, up5], axis=3)
    conv5 = Conv2D(128, 3, activation='relu', padding='same')(merge5)
    conv5 = Conv2D(128, 3, activation='relu', padding='same')(conv5)

    up6 = UpSampling2D(size=(2, 2))(conv5)
    merge6 = concatenate([conv2, up6], axis=3)
    conv6 = Conv2D(64, 3, activation='relu', padding='same')(merge6)
    conv6 = Conv2D(64, 3, activation='relu', padding='same')(conv6)

    up7 = UpSampling2D(size=(2, 2))(conv6)
    merge7 = concatenate([conv1, up7], axis=3)
    conv7 = Conv2D(32, 3, activation='relu', padding='same')(merge7)
    conv7 = Conv2D(32, 3, activation='relu', padding='same')(conv7)

    # 출력
    conv8 = Conv2D(1, (1, 1), activation='sigmoid')(conv7)

    model = Model(inputs=inputs, outputs=conv8)

    return model

unet = unet_model()
unet.compile(optimizer='adam',
             loss=BinaryCrossentropy(),
             metrics=['accuracy', Precision(), Recall(), AUC()])

In [9]:
early_stopping = EarlyStopping(monitor='val_loss',
                               patience=5,
                               verbose=1,
                               mode='min',
                               restore_best_weights=True)

checkpoint = ModelCheckpoint('cplab/model/dog_unet_model_{epoch:02d}.h5',
                             monitor='val_loss',
                             verbose=1,
                             save_best_only=False,
                             mode='min',
                             save_freq='epoch')

In [ ]:
history = unet.fit(train_dataset, epochs=10, steps_per_epoch=train_size // batch_size,
                   validation_data=val_dataset, validation_steps=val_size // batch_size,
                   callbacks=[checkpoint, early_stopping],
                   verbose=1)

Epoch 1/10
 538/3000 [====>.........................] - ETA: 2:23:41 - loss: 0.5751 - accuracy: 1.0000 - precision: 0.0000e+00 - recall: 0.0000e+00 - auc: 0.0000e+00

In [ ]:
plt.figure(figsize=(18, 4))

# 훈련 및 검증 손실 그래프
plt.subplot(1, 4, 1)
plt.plot(history.history['loss'], label='Training Loss')
plt.plot(history.history['val_loss'], label='Validation Loss')
plt.title('Training and Validation Loss')
plt.xlabel('Epoch')
plt.ylabel('Loss')
plt.legend()

# 훈련 및 검증 정확도 그래프
plt.subplot(1, 4, 2)
plt.plot(history.history['accuracy'], label='Training Accuracy')
plt.plot(history.history['val_accuracy'], label='Validation Accuracy')
plt.title('Training and Validation Accuracy')
plt.xlabel('Epoch')
plt.ylabel('Accuracy')
plt.legend()

# 훈련 및 검증 정밀도 그래프
plt.subplot(1, 4, 3)
plt.plot(history.history['precision'], label='Training Precision')
plt.plot(history.history['val_precision'], label='Validation Precision')
plt.title('Training and Validation Precision')
plt.xlabel('Epoch')
plt.ylabel('Precision')
plt.legend()

# 훈련 및 검증 재현율 그래프
plt.subplot(1, 4, 4)
plt.plot(history.history['recall'], label='Training Recall')
plt.plot(history.history['val_recall'], label='Validation Recall')
plt.title('Training and Validation Recall')
plt.xlabel('Epoch')
plt.ylabel('Recall')
plt.legend()

plt.tight_layout()
plt.show()
